In [15]:
import os
import pandas as pd

# Get current working directory (where your notebook runs)
cwd = os.getcwd()
print("Current working directory:", cwd)

# Build full paths to each file
input_folder = os.path.join(cwd, "input")

file_emps = os.path.join(input_folder, "hired_employees__1___1_.csv").replace("/notebooks","")
file_jobs = os.path.join(input_folder, "jobs__1__1.csv").replace("/notebooks","")
file_deps = os.path.join(input_folder, "departments__1___1_.csv").replace("/notebooks","")

# proj_de_scala_training/input/departments__1___1_.csv
# proj_de_scala_training/input/hired_employees__1___1_.csv
# proj_de_scala_training/input/jobs__1__1.csv
print(file_emps)


Current working directory: /home/jrob/proj_de_scala_training/notebooks
/home/jrob/proj_de_scala_training/input/hired_employees__1___1_.csv


In [19]:
import pandas as pd
from pathlib import Path

# Step 1: Load the CSV files
df_emps = pd.read_csv(file_emps, header=None,
                      names=["emp_id", "emp_name", "hire_date", "dept_id", "job_id"])

df_deps = pd.read_csv(file_deps, header=None, names=["dept_id", "dept_name"])

df_jobs = pd.read_csv(file_jobs, header=None, names=["job_id", "job_title"])


print(df_emps.head(3))
print(df_jobs.head(3))
print(df_deps.head(3))


   emp_id     emp_name             hire_date  dept_id  job_id
0       1  Harold Vogt  2021-11-07T02:48:42Z      2.0    96.0
1       2     Ty Hofer  2021-05-30T05:43:46Z      8.0     NaN
2       3  Lyman Hadye  2021-09-01T23:27:38Z      5.0    52.0
   job_id            job_title
0       1  Marketing Assistant
1       2             VP Sales
2       3   Biostatistician IV
   dept_id                 dept_name
0        1        Product Management
1        2                     Sales
2        3  Research and Development


In [23]:
# Step 2: Merge employee data with jobs and departments
df_merged = df_emps.merge(df_jobs, on="job_id", how="left") \
                   .merge(df_deps, on="dept_id", how="left")

print(df_merged.head(3))


   emp_id     emp_name             hire_date  dept_id  job_id  \
0       1  Harold Vogt  2021-11-07T02:48:42Z      2.0    96.0   
1       2     Ty Hofer  2021-05-30T05:43:46Z      8.0     NaN   
2       3  Lyman Hadye  2021-09-01T23:27:38Z      5.0    52.0   

                      job_title    dept_name  
0              Health Coach III        Sales  
1                           NaN      Support  
2  Structural Analysis Engineer  Engineering  


In [24]:
# Step 3: Cleanup missing values
df_merged['job_id'] = df_merged['job_id'].fillna(0).astype(int)
df_merged['dept_id'] = df_merged['dept_id'].fillna(0).astype(int)
df_merged['job_title'] = df_merged['job_title'].fillna("Not Found")
df_merged['dept_name'] = df_merged['dept_name'].fillna("Not Found")
df_merged['hire_date'] = df_merged['hire_date'].fillna("1900-01-01")

In [25]:
# Step 4: Wrangling hire_date to datetime and extract year
df_merged['hire_date'] = pd.to_datetime(df_merged['hire_date'], errors='coerce')
df_merged['hire_year'] = df_merged['hire_date'].dt.year


In [29]:
# Step 5: Group by department and list those cases above the mean

df_2021 = df_merged[df_merged['hire_year'] == 2021]

dept_hire_counts = df_2021.groupby(['dept_id', 'dept_name'])['emp_id'].count().reset_index()
dept_hire_counts.rename(columns={'emp_id': 'total_hired'}, inplace=True)

# Step 6 Filter departments above the average
mean_hired = dept_hire_counts['total_hired'].mean()
df_above_avg = dept_hire_counts[dept_hire_counts['total_hired'] > mean_hired]

df_above_avg_sorted = df_above_avg.sort_values(by='total_hired', ascending=False)
print(df_above_avg_sorted.head(5))

   dept_id             dept_name  total_hired
8        8               Support          221
5        5           Engineering          208
6        6       Human Resources          204
7        7              Services          204
4        4  Business Development          187


In [31]:
# Step 6: Calculate the Date columns

df_merged['hire_dd'] = pd.to_datetime(df_merged['hire_date'], errors='coerce')

df_merged['hire_qq'] = df_merged['hire_dd'].dt.quarter
df_merged['hire_yy'] = df_merged['hire_dd'].dt.year
df_merged['hire_mm'] = df_merged['hire_dd'].dt.month

df_merged['hire_period'] = df_merged['hire_yy'] * 100 + df_merged['hire_mm']

# Show first 5 rows
print(df_merged.head())

   emp_id        emp_name                 hire_date  dept_id  job_id  \
0       1     Harold Vogt 2021-11-07 02:48:42+00:00        2      96   
1       2        Ty Hofer 2021-05-30 05:43:46+00:00        8       0   
2       3     Lyman Hadye 2021-09-01 23:27:38+00:00        5      52   
3       4   Lotti Crowthe 2021-10-01 13:04:21+00:00       12      71   
4       5  Gretna Lording 2021-10-10 22:22:17+00:00        6      80   

                      job_title        dept_name  hire_year  \
0              Health Coach III            Sales     2021.0   
1                     Not Found          Support     2021.0   
2  Structural Analysis Engineer      Engineering     2021.0   
3               Statistician II       Accounting     2021.0   
4              Quality Engineer  Human Resources     2021.0   

                    hire_dd  hire_qq  hire_yy  hire_mm  hire_period  
0 2021-11-07 02:48:42+00:00      4.0   2021.0     11.0     202111.0  
1 2021-05-30 05:43:46+00:00      2.0   2021.0   

In [37]:
# Step 7: Pivot the Quarters per Deparment

grouped = df_merged.groupby(['dept_name', 'hire_qq'])['emp_id'].count().reset_index(name='total_hired')

pivot_df = grouped.pivot(index='dept_name', columns='hire_qq', values='total_hired')

pivot_df.columns = [f"Q{int(c)}" for c in pivot_df.columns]
pivot_df['total_hired_cases'] = pivot_df.sum(axis=1)

pivot_df = pivot_df.fillna(0).astype(int)
df_pivot_sorted = pivot_df.sort_values(by='total_hired_cases', ascending=False)

print(df_pivot_sorted.head(5))

                      Q1  Q2  Q3  Q4  total_hired_cases
dept_name                                              
Support               67  66  59  62                254
Human Resources       71  56  48  71                246
Engineering           63  47  61  73                244
Services              64  67  57  50                238
Business Development  59  58  49  53                219
